In [ ]:
import time
import rtsvg
rt = rtsvg.RACETrack()
from xwords import XWords
import os
_dir_    = '../../../data/crossword_puzzle_screenshots/'
_files_  = os.listdir(_dir_)
entries_file    = None
geometries_file = None
blockers_file   = None
for _file_ in _files_:
    if   'entries'    in _file_: entries_file    = _dir_ + _file_
    elif 'geometries' in _file_: geometries_file = _dir_ + _file_
    elif 'blockers'   in _file_: blockers_file   = _dir_ + _file_
xwords = XWords(rt, entries_file, geometries_file, blockers_file)
xwords

In [ ]:
from ollama import chat
from ollama import ChatResponse
from pydantic import BaseModel
model = 'gemma3:4b' # 'qwq' # 'deepseek-r1:32b' # 'gemma3:12b' # 'llama3.2'
def promptModel(prompt):
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content': prompt,},],)
    return response['message']['content']
promptModel('What is 55*3?  Return a single number.')

class Guess(BaseModel):
    guess: str

response_lu = {}

for cluenum, orientation in xwords.allClueNumbersAndOrientations():
    _tuple_ = (cluenum, orientation)
    if _tuple_ not in response_lu: response_lu[_tuple_] = []
    clue    = xwords.clue(cluenum, orientation)
    prompt  = f'Solve the crossword puzzle clue "{clue}" that is {xwords.numberOfLetters(cluenum, orientation)} letters long.  Return a single word in a JSON object.'
    t0 = time.time()
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content':  prompt,},], format=Guess.model_json_schema())
    t1 = time.time()
    response_lu[_tuple_].append((t1-t0, response))
    guess = Guess.model_validate_json(response['message']['content'])
    if len(guess.guess) != xwords.numberOfLetters(cluenum, orientation): 
        print('!',end='')
    else:
        xwords.guess(cluenum, orientation, guess.guess)
        print('.',end='')


In [ ]:
xwords

In [ ]:
xwords.clue(1, 'across'), xwords.clue(1, 'down')

In [ ]:
xwords.crossCluesAtCellCoordinates(12, 17)